In [1]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
import sys
sys.path.append('../../Python/')
from Preprocessing import standardize
from Evaluation import split_train_test_ma, train_model, test_model
from Databases import get_databases_path, get_database

In [3]:
def diff1(l, s, Nfre, N):
    epsilon = (0.03*N)/Nfre
    return (l-s) < epsilon 

def estimateThreshold(Freq_Table, Nfre, Frepos, N):
    maximaset = [1]
    minimaset = [1]
                
    table_keys = list(Freq_Table.keys())
    for idx in range(1, len(table_keys)-1):
        a = Freq_Table[table_keys[idx]]['Count'] - Freq_Table[table_keys[idx-1]]['Count'] 
        b = Freq_Table[table_keys[idx+1]]['Count'] - Freq_Table[table_keys[idx]]['Count'] 
        
        c1 = diff1(Frepos[idx], Frepos[(minimaset[-1])], Nfre, N)
        if a >=0 and b<=0 and c1:
            maximaset.append(idx)
        
        c2 = diff1(Frepos[(maximaset[-1])], Frepos[idx], Nfre, N)
        if a <=0 and b>=0 and c2:
            minimaset.append(idx)
    
    # Primer Bloque
    aux = np.zeros(len(maximaset))
    for idx in range(0, len(maximaset)):
        index = maximaset[idx]
        if Frepos[index] < 0.5:
            aux[idx] = Freq_Table[str(Frepos[index])]['Count']
        else: 
            aux[idx] = 0;
    
    if aux.sum() == 0:
        P1 = []
    else:
        amax = np.argmax(aux)
        P1 = maximaset[amax]
        fP1 = Frepos[P1]
     
    # Segundo Bloque
    aux = np.zeros(len(maximaset))
    for idx in range(0, len(maximaset)):
        index = maximaset[idx]
        if Frepos[index] > fP1:
            aux[idx] = Freq_Table[str(Frepos[index])]['Count']
        else: 
            aux[idx] = 0;

    if aux.sum() == 0:
        P2 = []
    else:
        amax = np.argmax(aux)
        P2 = maximaset[amax]
        fP2 = Frepos[P2]
        

    if P1 is not None and P2 is not None:
        if not minimaset:
            valley = [];
        else:
            aux = np.zeros(len(minimaset))
            for idx in range(0, len(minimaset)):
                index = minimaset[idx]
                if Frepos[index] > fP1 and Frepos[index] < fP2: 
                    aux[idx] = Freq_Table[str(Frepos[index])]['Count']
                else: 
                    aux[idx] = 0
            
            if aux.sum() == 0:
                valley = []
            else:
                amax = np.argmax(aux)
                valley = maximaset[amax]
                fvalley = Frepos[valley]
    else:
        valley = [];
        
    if not valley:
        t = P1
    else:
        t = valley
    
    Nl = 0
    for idx in range(0,t):
        Nl += Freq_Table[table_keys[idx]]['Count']

    while Nl < N/2:
        t += 1;
        Nl += Freq_Table[table_keys[t]]['Count']
    Nr = N - Nl
    return t, Nl, Nr

In [12]:
databases_path = '../../Databases/Sinteticas'
paths = get_databases_path(databases_path)

In [13]:
# Obtain the attributes and labels
attr_df = get_database(paths[0], 'database_attr.csv')
labels_df = get_database(paths[0], 'database_labels.csv')

In [14]:
# Preprocessing the attributes
standardized_attr_df = standardize(attr_df)

In [15]:
# Split in Training and Test Sets
X_train, X_test, y_train, y_test = split_train_test_ma(standardized_attr_df, labels_df)

In [16]:
R = y_train.columns.size
N = y_train.shape[0]

step = 1/R
Frepos = np.arange(0, 1+step, step)
Nfre = len(Frepos)

In [17]:
Freq_Table = {}
for key in Frepos:
    index = str(round(key,1))
    Freq_Table[index] = {}
    Freq_Table[index]['Count'] = 0
    Freq_Table[index]['Items'] = []
    Freq_Table[index]['Cat'] = 0
table_keys = list(Freq_Table.keys())

In [18]:
for idx, row in y_train.iterrows():
    fi = round(row[row == True].sum()/R,1)
    index = str(round(fi,1))
    Freq_Table[index]['Count'] += 1
    Freq_Table[index]['Items'].append(idx)

In [19]:
t, Nl, Nr = estimateThreshold(Freq_Table, Nfre, Frepos, N)

In [20]:
Pmax = (Nl - Nr)*Nr/(Nl + Nr) + Nr;
k = Nfre-1;
Np = 0;
while k > t:
    Freq_Table[table_keys[k]]['Cat'] = 1;
    Np += Freq_Table[table_keys[k]]['Count']
    k -= 1

In [21]:
fm = (Frepos[0] + Frepos[t])*0.5
k = t
while (Frepos[k] > fm) and (Np + Freq_Table[table_keys[k]]['Count'] < Pmax):
    Freq_Table[table_keys[k]]['Cat'] = 1
    Np += Freq_Table[table_keys[k]]['Count']
    k -= 1

In [22]:
hatY = np.zeros((N,1))
for idx in range(0,Nfre):
    aux = Freq_Table[table_keys[idx]]['Items'];
    y_train.loc[aux, 'yHat'] = Freq_Table[table_keys[idx]]['Cat']
yHat = y_train[['yHat']].astype('bool')

In [23]:
model= train_model(X_train, yHat)

In [24]:
test_model(model, X_test, y_test)

Global Precision:
0.8654970760233918


General Report:
              precision    recall  f1-score   support

         yes       0.88      0.91      0.89       108
          no       0.83      0.79      0.81        63

    accuracy                           0.87       171
   macro avg       0.86      0.85      0.85       171
weighted avg       0.86      0.87      0.86       171



Confusion Matrix: 
    0   1
0  98  10
1  13  50
